In [ ]:
import psycopg2
import pandas as pd
import logging
import sys
import os
from pathlib import Path

# 현재 노트북 파일의 상위 디렉토리에 있는 src 디렉토리의 경로를 sys.path에 추가
module_path = Path('../src').resolve()
if module_path not in sys.path:
    sys.path.append(str(module_path))

# Logging config
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 소스코드(src)
import src.data_extraction.access_database as db
import src.data_extraction.filter_adult_patients as fap
import src.data_extraction.filter_ventilation_events as fve
from src.utils import utils


In [ ]:
# 데이터 저장위치
output_dir = './data'


# 데이터베이스 연결 설정을 하나의 딕셔너리로 통합
db_config = {
    'database': 'mimiciv',
    'user': 'mai_onlyselect',
    'password': 'student1q2w!@',
    'host': '1.212.63.162',
    'port': '35430'
}

# 호흡기 ITEM ID 설정
vent_ids_config = {
    'INTUBATION_ITEM_IDS': "224385",
    'EXTUBATION_ITEM_IDS': "225468, 225477, 227194"
}

# 테이블 쿼리 설정
tables_query = {
    'pg_tables': 'SELECT * FROM PG_TABLES;',
    'patients': 'SELECT * FROM mimiciv_hosp.patients;',
    'admissions': 'SELECT * FROM mimiciv_hosp.admissions;',
    'transfers': 'SELECT * FROM mimiciv_hosp.transfers;',
    'icustays': 'SELECT * FROM mimiciv_icu.icustays;',
    'd_items': 'SELECT * FROM mimiciv_icu.d_items;',
    'intubation': f"SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN ({vent_ids_config['INTUBATION_ITEM_IDS']});",
    'extubation': f"SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN ({vent_ids_config['EXTUBATION_ITEM_IDS']});",
    'ventilation': 'SELECT * FROM mimiciv_derived.ventilation;'
}

# 데이터베이스 연결 함수
def connect_to_database(config):
    try:
        conn = psycopg2.connect(**config)
        print("Connected to the database successfully!")
        return conn
    except (psycopg2.Error, Exception) as error:
        print("Error while connecting to the database:", error)
        return None

# 데이터베이스 연결
conn = connect_to_database(db_config)

if conn is not None:
    # 커서 생성
    cur = conn.cursor()

    # 데이터베이스 작업 수행
    dataframes = db.retrieve_data(conn, tables_query)

    # PostgreSQL 버전 확인
    cur.execute("SELECT version();")
    version = cur.fetchone()
    print("PostgreSQL version:", version)

    # 연결 종료는 필요한 시점에 수행
    # cur.close()
    # conn.close()
else:
    print("Failed to connect to the database.")
